<a href="https://colab.research.google.com/github/andrebelem/pythonverse/blob/main/Extrator_Esta%C3%A7%C3%B5es_Hist%C3%B3ricas_INMET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extrator de dados do INMET

Criado por [Andre L. Belém](mailto:andrebelem@id.uff.br)

Histórico: Março de 2024<br>
A biblioteca `hydrobr` deixou de funcionar logo após uma série de ataques "hacker" ao Instituto Nacional de Meteorologia (INMET). Por esse motivo, tive que extrair os dados da estação meteorológica de Uruguaiana (A809) manualmente a partir da página [https://portal.inmet.gov.br/dadoshistoricos](https://portal.inmet.gov.br/dadoshistoricos).

Este script realiza essa tarefa. É importante notar que ele pode ser utilizado para qualquer estação do INMET na base histórica.

In [2]:
# esta operação de baixar os dados e serpará-los demora em média 6 a 8 minutos.
import requests
import os
from tqdm import tqdm
import zipfile
import shutil

dest_dir = "dados_inmet"

if not os.path.exists(dest_dir):
    os.makedirs(dest_dir)

for year in tqdm(range(2000, 2025), desc="Baixando arquivos do INMET "):
    url = f"https://portal.inmet.gov.br/uploads/dadoshistoricos/{year}.zip"
    zip_path = os.path.join(dest_dir, f"{year}.zip")
    with open(zip_path, 'wb') as f:
        f.write(requests.get(url).content)
    zipfile.ZipFile(zip_path, 'r').extractall(os.path.join(dest_dir, str(year)))

print("Downloads concluídos.")

Baixando arquivos do INMET : 100%|██████████| 25/25 [06:29<00:00, 15.58s/it]

Downloads concluídos.


In [3]:
# conta o número de arquivos
csv_count = sum(1 for root, dirs, files in os.walk(dest_dir) for file in files if file.lower().endswith('.csv'))

print("Numero de arquivos na pasta:", csv_count)


Numero de arquivos na pasta: 9460


In [4]:
# separa a estação "A809"
files_sta = [os.path.join(root, file) for root, dirs, files in os.walk(dest_dir) for file in files if 'A809' in file]

print("Arquivos da A809:")
for file_path in files_sta:
    print(file_path)

Arquivos da A809:
dados_inmet/2011/2011/INMET_S_RS_A809_URUGUAIANA_01-01-2011_A_31-12-2011.CSV
dados_inmet/2018/2018/INMET_S_RS_A809_URUGUAIANA_01-01-2018_A_31-12-2018.CSV
dados_inmet/2022/INMET_S_RS_A809_URUGUAIANA_01-01-2022_A_31-12-2022.CSV
dados_inmet/2010/2010/INMET_S_RS_A809_URUGUAIANA_01-01-2010_A_31-12-2010.CSV
dados_inmet/2019/2019/INMET_S_RS_A809_URUGUAIANA_01-01-2019_A_31-12-2019.CSV
dados_inmet/2024/INMET_S_RS_A809_URUGUAIANA_01-01-2024_A_29-02-2024.CSV
dados_inmet/2021/INMET_S_RS_A809_URUGUAIANA_01-01-2021_A_31-12-2021.CSV
dados_inmet/2014/2014/INMET_S_RS_A809_URUGUAIANA_01-01-2014_A_31-12-2014.CSV
dados_inmet/2013/2013/INMET_S_RS_A809_URUGUAIANA_01-01-2013_A_31-12-2013.CSV
dados_inmet/2017/2017/INMET_S_RS_A809_URUGUAIANA_01-01-2017_A_31-12-2017.CSV
dados_inmet/2020/INMET_S_RS_A809_URUGUAIANA_01-01-2020_A_31-12-2020.CSV
dados_inmet/2016/2016/INMET_S_RS_A809_URUGUAIANA_01-01-2016_A_31-12-2016.CSV
dados_inmet/2015/2015/INMET_S_RS_A809_URUGUAIANA_01-01-2015_A_31-12-2015.CSV
d

Aqui vou ler os arquivos da A809 e consertar pequenos erros de formato entre os anos. Sempre verifique a consistência dos dados pois aparentemente não há controle de qualidade destes dados.

In [17]:
import pandas as pd
from tqdm import tqdm

df_sta = pd.DataFrame()

# Adicionando um contador tqdm
for i, file in tqdm(enumerate(files_sta),desc=f"Lendo arquivos...", total=len(files_sta)):
    #print(f'lendo arquivo {file}')
    df = pd.read_csv(file, encoding='latin1', skiprows=8, sep=';', decimal=",")
    df.columns.values[:2] = ['DATA', 'HORA_UTC']  # compatibiliza a coluna de data e hora
                                                  # porque em 2023 muda
    df.columns.values[6] = 'RADIACAO GLOBAL (KJ/m²)'  # existe uma mudança no nome em alguns anos
                                                      # esta linha acerta essas incoerencias
    df_sta = pd.concat([df_sta, df], ignore_index=True)


Lendo arquivos...: 100%|██████████| 19/19 [00:00<00:00, 43.24it/s]


In [18]:
# converte tudo para o mesmo datetime e realinha
import numpy as np
# Concatena as colunas DATA e HORA em uma única coluna
df_sta['time'] = pd.to_datetime(df_sta['DATA'] + ' ' + df_sta['HORA_UTC'],errors='coerce')
df_sta['time'] = pd.to_datetime(df_sta['time'],utc=True)

df_sta = df_sta.sort_values(by='time')
# reposiociona na primera coluna
df_sta.insert(0, 'time', df_sta.pop('time'))

In [21]:
# agora converte os valors -9999 em nan
df_sta.iloc[:, 3:] = df_sta.iloc[:, 3:].applymap(lambda x: np.nan if x == -9999 else float(x))

In [23]:
# acerta o resto das colunas e reseta o index.
df_sta = df_sta.reset_index(drop=True).drop(columns='Unnamed: 19')

,time,DATA,HORA_UTC,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (KJ/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)"
0,2006-09-28 00:00:00+00:00,2006-09-28,00:00,1.2,1007.5,1008.7,1007.3,NaN,17.8,17.3,18.4,17.8,17.6,17.3,97.0,95.0,97.0,137.0,7.4,3.2
1,2006-09-28 01:00:00+00:00,2006-09-28,01:00,1.0,1010.5,1010.5,1007.5,NaN,18.1,17.5,18.2,17.8,17.6,17.3,97.0,95.0,96.0,221.0,6.4,2.1
2,2006-09-28 02:00:00+00:00,2006-09-28,02:00,5.8,1009.7,1010.5,1009.7,NaN,17.6,17.0,18.2,17.6,17.5,16.9,96.0,96.0,96.0,144.0,6.9,4.0
3,2006-09-28 03:00:00+00:00,2006-09-28,03:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2006-09-28 04:00:00+00:00,2006-09-28,04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152731,2024-02-29 19:00:00+00:00,2024/02/29,1900 UTC,0.0,1003.3,1003.9,1003.3,1655.1,29.6,22.9,30.4,29.5,24.2,22.9,72.0,66.0,67.0,309.0,7.1,2.8
152732,2024-02-29 20:00:00+00:00,2024/02/29,2000 UTC,0.0,1003.1,1003.3,1003.0,1634.3,30.7,23.2,30.8,29.6,24.0,22.8,69.0,64.0,64.0,291.0,5.1,1.7
152733,2024-02-29 21:00:00+00:00,2024/02/29,2100 UTC,0.0,1003.2,1003.3,1003.0,1282.1,29.9,23.0,30.8,29.8,23.4,22.1,67.0,62.0,66.0,313.0,4.0,1.7
152734,2024-02-29 22:00:00+00:00,2024/02/29,2200 UTC,0.0,1003.8,1003.8,1003.2,385.2,27.6,22.8,29.9,27.6,23.2,22.4,75.0,66.0,75.0,111.0,6.8,3.4


In [24]:
# agora por último grava em parquet, para ser usado depois
df_sta.to_parquet('INMET_A809_Historical.parquet')

Pronto. Com este mecanismo "simples" você já tem acesso aos dados históricos de uma estação do INMET. Vamos extrair mais uma ? Que tal a estação A652 do Forte de Copacabana no Rio de Janeiro ?!

In [25]:
# separa a estação "A652"
files_sta = [os.path.join(root, file) for root, dirs, files in os.walk(dest_dir) for file in files if 'A652' in file]

print("Arquivos da A652:")
for file_path in files_sta:
    print(file_path)

Arquivos da A652:
dados_inmet/2011/2011/INMET_SE_RJ_A652_FORTE DE COPACABANA_01-01-2011_A_31-12-2011.CSV
dados_inmet/2018/2018/INMET_SE_RJ_A652_FORTE DE COPACABANA_01-01-2018_A_31-12-2018.CSV
dados_inmet/2022/INMET_SE_RJ_A652_RIO DE JANEIRO - FORTE DE COPACABANA_01-01-2022_A_31-12-2022.CSV
dados_inmet/2010/2010/INMET_SE_RJ_A652_FORTE DE COPACABANA_01-01-2010_A_31-12-2010.CSV
dados_inmet/2019/2019/INMET_SE_RJ_A652_RIO DE JANEIRO - FORTE DE COPACABANA_01-01-2019_A_31-12-2019.CSV
dados_inmet/2024/INMET_SE_RJ_A652_RIO DE JANEIRO - FORTE DE COPACABANA_01-01-2024_A_29-02-2024.CSV
dados_inmet/2021/INMET_SE_RJ_A652_RIO DE JANEIRO - FORTE DE COPACABANA_01-01-2021_A_31-12-2021.CSV
dados_inmet/2014/2014/INMET_SE_RJ_A652_FORTE DE COPACABANA_01-01-2014_A_31-12-2014.CSV
dados_inmet/2013/2013/INMET_SE_RJ_A652_FORTE DE COPACABANA_01-01-2013_A_31-12-2013.CSV
dados_inmet/2017/2017/INMET_SE_RJ_A652_FORTE DE COPACABANA_01-01-2017_A_31-12-2017.CSV
dados_inmet/2020/INMET_SE_RJ_A652_RIO DE JANEIRO - FORTE DE

In [26]:
import pandas as pd
from tqdm import tqdm

df_sta = pd.DataFrame()

# Adicionando um contador tqdm
for i, file in tqdm(enumerate(files_sta),desc=f"Lendo arquivos...", total=len(files_sta)):
    #print(f'lendo arquivo {file}')
    df = pd.read_csv(file, encoding='latin1', skiprows=8, sep=';', decimal=",")
    df.columns.values[:2] = ['DATA', 'HORA_UTC']  # compatibiliza a coluna de data e hora
                                                  # porque em 2023 muda
    df.columns.values[6] = 'RADIACAO GLOBAL (KJ/m²)'  # existe uma mudança no nome em alguns anos
                                                      # esta linha acerta essas incoerencias
    df_sta = pd.concat([df_sta, df], ignore_index=True)

# converte tudo para o mesmo datetime e realinha
import numpy as np
# Concatena as colunas DATA e HORA em uma única coluna
df_sta['time'] = pd.to_datetime(df_sta['DATA'] + ' ' + df_sta['HORA_UTC'],errors='coerce')
df_sta['time'] = pd.to_datetime(df_sta['time'],utc=True)

df_sta = df_sta.sort_values(by='time')
# reposiociona na primera coluna
df_sta.insert(0, 'time', df_sta.pop('time'))

# agora converte os valors -9999 em nan
df_sta.iloc[:, 3:] = df_sta.iloc[:, 3:].applymap(lambda x: np.nan if x == -9999 else float(x))
# acerta o resto das colunas e reseta o index.
df_sta = df_sta.reset_index(drop=True).drop(columns='Unnamed: 19')

# agora por último grava em parquet, para ser usado depois
df_sta.to_parquet('INMET_A652_Historical.parquet')

Lendo arquivos...: 100%|██████████| 18/18 [00:00<00:00, 32.11it/s]


In [31]:
# veja o resultado aqui
df_sta.tail(5)

,time,DATA,HORA_UTC,"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB),PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB),RADIACAO GLOBAL (KJ/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C),TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C),TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C),UMIDADE REL. MAX. NA HORA ANT. (AUT) (%),UMIDADE REL. MIN. NA HORA ANT. (AUT) (%),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, DIREÇÃO HORARIA (gr) (° (gr))","VENTO, RAJADA MAXIMA (m/s)","VENTO, VELOCIDADE HORARIA (m/s)"
147163,2024-02-29 19:00:00+00:00,2024/02/29,1900 UTC,0.0,1007.2,1007.2,1006.8,NaN,29.3,26.5,29.3,27.9,26.5,25.1,87.0,84.0,85.0,235.0,5.2,2.6
147164,2024-02-29 20:00:00+00:00,2024/02/29,2000 UTC,0.0,1007.1,1007.3,1007.1,NaN,28.6,25.8,29.6,28.4,26.7,25.6,86.0,84.0,85.0,231.0,5.2,1.9
147165,2024-02-29 21:00:00+00:00,2024/02/29,2100 UTC,0.0,1006.9,1007.2,1006.8,NaN,27.2,25.7,28.6,27.2,25.9,25.6,92.0,85.0,92.0,249.0,4.9,2.7
147166,2024-02-29 22:00:00+00:00,2024/02/29,2200 UTC,0.0,1007.4,1007.4,1006.9,NaN,27.1,25.6,27.2,26.9,25.8,25.5,93.0,91.0,92.0,256.0,4.9,2.3
147167,2024-02-29 23:00:00+00:00,2024/02/29,2300 UTC,0.0,1008.0,1008.2,1007.4,NaN,26.9,25.6,27.1,26.8,25.8,25.5,93.0,91.0,93.0,267.0,4.8,1.5


Note que há inúmeras possibilidades aqui ! Aproveite para explorá-las.